In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# CUDA setup

In [ ]:
!nvcc --version

In [ ]:
!nvidia-smi

## NVCC Plugin for Jupyter notebook

*Usage*:


*   Load Extension `%load_ext nvcc_plugin`
*   Mark a cell to be treated as cuda cell
`%%cuda --name example.cu --compile false`

**NOTE**: The cell must contain either code or comments to be run successfully. It accepts 2 arguments. `-n | --name` - which is the name of either CUDA source or Header. The name parameter must have extension `.cu` or `.h`. Second argument -c | --compile; default value is false. The argument is a flag to specify if the cell will be compiled and run right away or not. It might be usefull if you're playing in the main function

*  We are ready to run CUDA C/C++ code right in your Notebook. For this we need explicitly say to the interpreter, that we want to use the extension by adding `%%cu` at the beginning of each cell with CUDA code. 




In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

# CD

In [ ]:
%cd /content/drive/MyDrive/GPU_Computing_Project
%ls

#DeviceQuery

In [ ]:
#@title working directory: **deviceQuery/**
%cd /content/drive/MyDrive/GPU_Computing_Project/deviceQuery
%ls

In [ ]:
%%writefile ./helper.h
// Beginning of GPU Architecture definitions
inline int _ConvertSMVer2Cores(int major, int minor) {
	// Defines for GPU Architecture types (using the SM version to determine
	// the # of cores per SM
	typedef struct {
		int SM;  // 0xMm (hexidecimal notation), M = SM Major version,
		// and m = SM minor version
		int Cores;
	} sSMtoCores;

	sSMtoCores nGpuArchCoresPerSM[] = {
			{0x20, 32},
			{0x30, 192},
			{0x32, 192},
			{0x35, 192},
			{0x37, 192},
			{0x50, 128},
			{0x52, 128},
			{0x53, 128},
			{0x60,  64},
			{0x61, 128},
			{0x62, 128},
			{0x70,  64},
			{0x72,  64},
			{0x75,  64},
			{-1, -1}};

	int index = 0;

	while (nGpuArchCoresPerSM[index].SM != -1) {
		if (nGpuArchCoresPerSM[index].SM == ((major << 4) + minor)) {
			return nGpuArchCoresPerSM[index].Cores;
		}

		index++;
	}

	// If we don't find the values, we default use the previous one
	// to run properly
	printf(
			"MapSMtoCores for SM %d.%d is undefined."
			"  Default to use %d Cores/SM\n",
			major, minor, nGpuArchCoresPerSM[index - 1].Cores);
	return nGpuArchCoresPerSM[index - 1].Cores;
}


In [ ]:
%%writefile ./deviceQuery.cu
#include <stdlib.h>
#include <stdio.h>
#include "helper.h"
#include "../utils/common.h"

int main(void) {

	printf("\nCUDA Device Query (Runtime API) version (CUDART static linking)\n\n");
	int deviceCount = 0;
	CHECK(cudaGetDeviceCount(&deviceCount));

	// This function call returns 0 if there are no CUDA capable devices.
	if (deviceCount == 0)
		printf("There are no available device(s) that support CUDA\n");
	else
		printf("Detected %d CUDA Capable device(s)\n", deviceCount);

	int dev, driverVersion = 0, runtimeVersion = 0;

	for (dev = 0; dev < deviceCount; ++dev) {
		cudaSetDevice(dev);
		cudaDeviceProp deviceProp;
		cudaGetDeviceProperties(&deviceProp, dev);

		printf("\nDevice %d: \"%s\"\n", dev, deviceProp.name);

		cudaDriverGetVersion(&driverVersion);
		cudaRuntimeGetVersion(&runtimeVersion);

		printf("  CUDA Driver Version / Runtime Version          %d.%d / %d.%d\n",
				driverVersion / 1000, (driverVersion % 100) / 10,
				runtimeVersion / 1000, (runtimeVersion % 100) / 10);

		printf("  CUDA Capability Major/Minor version number:    %d.%d\n",
				deviceProp.major, deviceProp.minor);

		printf("  Total amount of global memory:                 %.0f MBytes (%llu bytes)\n",
				(float) deviceProp.totalGlobalMem / 1048576.0f,
				(unsigned long long) deviceProp.totalGlobalMem);

	    printf("  (%2d) Multiprocessors, (%3d) CUDA Cores/MP:     %d CUDA Cores\n",
	           deviceProp.multiProcessorCount,
	           _ConvertSMVer2Cores(deviceProp.major, deviceProp.minor),
	           _ConvertSMVer2Cores(deviceProp.major, deviceProp.minor) *
	               deviceProp.multiProcessorCount);

		printf("  GPU Max Clock rate:                            %.0f MHz (%0.2f GHz)\n",
				deviceProp.clockRate * 1e-3f, deviceProp.clockRate * 1e-6f);

		printf("  Memory Clock rate:                             %.0f Mhz\n", deviceProp.memoryClockRate * 1e-3f);
		printf("  Memory Bus Width:                              %d-bit\n", deviceProp.memoryBusWidth);
		if (deviceProp.l2CacheSize)
			printf("  L2 Cache Size:                                 %d bytes\n", deviceProp.l2CacheSize);

		printf("  Maximum Texture Dimension Size (x,y,z)         1D=(%d), 2D=(%d, %d), 3D=(%d, %d, %d)\n",
				deviceProp.maxTexture1D, deviceProp.maxTexture2D[0],
				deviceProp.maxTexture2D[1], deviceProp.maxTexture3D[0],
				deviceProp.maxTexture3D[1], deviceProp.maxTexture3D[2]);

		printf("  Maximum Layered 1D Texture Size, (num) layers  1D=(%d), %d layers\n",
				deviceProp.maxTexture1DLayered[0],
				deviceProp.maxTexture1DLayered[1]);

		printf("  Maximum Layered 2D Texture Size, (num) layers  2D=(%d, %d), %d layers\n",
				deviceProp.maxTexture2DLayered[0],
				deviceProp.maxTexture2DLayered[1],
				deviceProp.maxTexture2DLayered[2]);

		printf("  Total amount of constant memory:               %lu bytes\n",
				deviceProp.totalConstMem);
		printf("  Total amount of shared memory per block:       %lu bytes\n",
				deviceProp.sharedMemPerBlock);
		printf("  Total number of registers available per block: %d\n",
				deviceProp.regsPerBlock);
		printf("  Warp size:                                     %d\n",
				deviceProp.warpSize);
		printf("  Maximum number of threads per multiprocessor:  %d\n",
				deviceProp.maxThreadsPerMultiProcessor);
		printf("  Maximum number of threads per block:           %d\n",
				deviceProp.maxThreadsPerBlock);
		printf("  Max dimension size of a thread block (x,y,z): (%d, %d, %d)\n",
				deviceProp.maxThreadsDim[0], deviceProp.maxThreadsDim[1],
				deviceProp.maxThreadsDim[2]);
		printf("  Max dimension size of a grid size    (x,y,z): (%d, %d, %d)\n",
				deviceProp.maxGridSize[0], deviceProp.maxGridSize[1],
				deviceProp.maxGridSize[2]);
		printf("  Maximum memory pitch:                          %lu bytes\n",
				deviceProp.memPitch);
		printf("  Texture alignment:                             %lu bytes\n",
				deviceProp.textureAlignment);
		printf("  Concurrent copy and kernel execution:          %s with %d copy engine(s)\n",
				(deviceProp.deviceOverlap ? "Yes" : "No"),
				deviceProp.asyncEngineCount);
		printf("  Run time limit on kernels:                     %s\n",
				deviceProp.kernelExecTimeoutEnabled ? "Yes" : "No");
		printf("  Integrated GPU sharing Host Memory:            %s\n",
				deviceProp.integrated ? "Yes" : "No");
		printf("  Support host page-locked memory mapping:       %s\n",
				deviceProp.canMapHostMemory ? "Yes" : "No");
		printf("  Alignment requirement for Surfaces:            %s\n",
				deviceProp.surfaceAlignment ? "Yes" : "No");
		printf("  Device has ECC support:                        %s\n",
				deviceProp.ECCEnabled ? "Enabled" : "Disabled");

		printf("  Device supports Unified Addressing (UVA):      %s\n",
				deviceProp.unifiedAddressing ? "Yes" : "No");
		printf("  Device PCI Domain ID / Bus ID / location ID:   %d / %d / %d\n",
				deviceProp.pciDomainID, deviceProp.pciBusID,
				deviceProp.pciDeviceID);
	}
	return 0;
}


In [ ]:
# Compilazione ed esecuzione

!nvcc deviceQuery.cu -o deviceQuery
!./deviceQuery

# WarpSort

Qua ci va l'introduzione


*   In cosa consiste l'algoritmo 
*   Come viene implementato
*   Caso sequenziale
*   Caso parallelo
*   Test e risultati
*   Conclusioni



In [ ]:
%cd /content/drive/MyDrive/GPU_Computing_Project/WarpSort/
%ls

In [ ]:
%%writefile ./WarpSort.cu
#include <stdlib.h>
#include <stdio.h>
#include <time.h>

#include "../utils/common.h"

#define THREADS 16
#define BLOCKS 1


/*
 * Indici k e j in input hanno significato:
 *     k = 2,4,8,...,2^s=N
 *     j = 2^(k-1), 2^(k-2),...,1 (parte dalla metà di k e continua a dimezzare)
 * Gli operatori sui bit ^ (XOR) e & (AND) vengono usati per filtrare i thread:
 *     ixj = i ^ j  aggiunge o toglie a i una potenza di 2, cioé ixj = i +- j (j = 2^a)
 *     i & k == 0   vero sse i <= k (sort ascendente) altrimenti sort discendente
 * L'operazione ixj > i significa aggiorna solo quando l'indice ixj fa un salto in
 * avanti di j = 2^a
 * Funzionamento:
 */
__global__ void bitonic_sort_step(int *a, int j, int k) {
	unsigned int i, ixj;                       // Sorting partners i and ixj
	i = threadIdx.x + blockDim.x * blockIdx.x;
	ixj = i ^ j;   // XOR: aggiunge o toglie a i una potenza di 2, j = 2^a

	if (i == 0)
		printf("ROUND: k = %d, j = %d\n", k, j);

	if ((ixj) > i) {    // entra solo quando fa un salto di j = 2^a

//		 Sort ascending
		if ((i & k) == 0) {
			printf("  UP  (ixj = %d\t    i = %d\t k = %d)   a[ixj] = %d - a[i] = %d\n", ixj, i, k, a[ixj],a[i]);
			if (a[i] > a[ixj]) {
				int temp = a[i];
				a[i] = a[ixj];
				a[ixj] = temp;
			}
		}

		// Sort descending
		if ((i & k) != 0) {
			printf("  DOWN  (ixj = %d\t    i = %d\t k = %d)   a[ixj] = %d - a[i] = %d\n", ixj, i, k, a[ixj],a[i]);
			if (a[i] < a[ixj]) {
				int temp = a[i];
				a[i] = a[ixj];
				a[ixj] = temp;
			}
		}
	}
}

__global__ void bitonic_sort_warp(int *keyin){
  //prendere thread id giusto tenendo in considerazione k_0 e k_1
  //implementare gli swap fatti bene dentro la funzione

  int i,j,k_0,k_1 = 0;
  //phase 0 to log(128)-1 
  for(i=2;i<128;i*=2){ 
    for(j=i/2;j>0;j/=2){ 
      //k_0 ? position of preceding element in each pair to form ascending order
      if(keyin[k_0]>keyin[k_0+j]) 
        swap(keyin[k_0],keyin[k_0+j]);
      //k1 ? position of preceding element in each pair to form descending order
      if(keyin[k_1]<keyin[k_1+j]) 
        swap(keyin[k_1],keyin[k_1+j]);
    }
  }

  //special case for the last phase 
  for(j=128/2;j>0;j/=2){ 
    //k0 ? position of preceding element in the thread's first pair to form ascending order
    if(keyin[k_0]>keyin[k_0+j]) 
      swap(keyin[k_0],keyin[k_0+j]);
    //k1 ? position of preceding element in the thread's second pair to form ascending order
    if(keyin[k_1]>keyin[k_1+j]) 
      swap(keyin[k_1],keyin[k_1+j]);
  }
}

/*The parameter dir indicates the sorting direction, ASCENDING
 or DESCENDING; if (a[i] > a[j]) agrees with the direction,
 then a[i] and a[j] are interchanged.*/
void compAndSwap(int a[], int i, int j, int dir) {
	if (dir == (a[i] > a[j])) {
		int tmp = a[i];
		a[i] = a[j];
		a[j] = tmp;
	}
}

/*It recursively sorts a bitonic sequence in ascending order,
 if dir = 1, and in descending order otherwise (means dir=0).
 The sequence to be sorted starts at index position low,
 the parameter cnt is the number of elements to be sorted.*/
void bitonicMerge(int a[], int low, int cnt, int dir) {
	if (cnt > 1) {
		int k = cnt / 2;
		for (int i = low; i < low + k; i++)
			compAndSwap(a, i, i + k, dir);
		bitonicMerge(a, low, k, dir);
		bitonicMerge(a, low + k, k, dir);
	}
}

/* This function first produces a bitonic sequence by recursively
 sorting its two halves in opposite sorting orders, and then
 calls bitonicMerge to make them in the same order */
void bitonicSort(int a[], int low, int cnt, int dir) {
	if (cnt > 1) {
		int k = cnt / 2;

		// sort in ascending order since dir here is 1
		bitonicSort(a, low, k, 1);

		// sort in descending order since dir here is 0
		bitonicSort(a, low + k, k, 0);

		// Will merge wole sequence in ascending order
		// since dir=1.
		bitonicMerge(a, low, cnt, dir);
	}
}

/*
 * test bitonic sort on CPU and GPU
 */
int main(void) {
	cudaEvent_t start, stop;
	cudaEventCreate(&start);
	cudaEventCreate(&stop);

	int N = THREADS*BLOCKS;
	// check
	if (!(N && !(N & (N - 1)))) {
		printf("ERROR: N must be power of 2 (N = %d)\n", N);
		exit(1);
	}
	size_t nBytes = N * sizeof(int);
	int *a = (int*) malloc(nBytes);
	int *b = (int*) malloc(nBytes);

	// fill data
	for (int i = 0; i < N; ++i) {
		a[i] =  i%5; //rand() % 100; // / (float) RAND_MAX;
		b[i] = a[i];
	}

	// bitonic CPU
	double cpu_time = seconds();
	bitonicSort(b, 0, N, 1);   // 1 means sort in ascending order
	printf("CPU elapsed time: %.5f (sec)\n", seconds()-cpu_time);

	// device mem copy
	int *d_a;
	CHECK(cudaMalloc((void**) &d_a, nBytes));
	CHECK(cudaMemcpy(d_a, a, nBytes, cudaMemcpyHostToDevice));

	// num of threads
	dim3 blocks(BLOCKS, 1);   // Number of blocks
	dim3 threads(THREADS, 1); // Number of threads

	// start computation
	cudaEventRecord(start);
	int j, k;
	// external loop on comparators of size k
	for (k = 2; k <= N; k <<= 1) {
		// internal loop for comparator internal stages
		for (j = k >> 1; j > 0; j = j >> 1)
			bitonic_sort_step<<<blocks, threads>>>(d_a, j, k);
	}

  /*Divide the input sequence into equal-sized subsequences. 
  Each subsequence will be sorted by an independent warp using the bitonic network.*/
  bitonic_sort_warp<<<blocks, threads>>>(d_a);

	cudaEventRecord(stop);
	cudaEventSynchronize(stop);
	float milliseconds = 0;
	cudaEventElapsedTime(&milliseconds, start, stop);
	printf("GPU elapsed time: %.5f (sec)\n", milliseconds / 1000);

	// recover data
	cudaMemcpy(a, d_a, nBytes, cudaMemcpyDeviceToHost);

	// print & check
	if (N < 100) {
		printf("GPU:\n");
		for (int i = 0; i < N; ++i)
			printf("%d\n", a[i]);
		printf("CPU:\n");
		for (int i = 0; i < N; ++i)
			printf("%d\n", b[i]);
	}
	else {
		for (int i = 0; i < N; ++i) {
			if (a[i] != b[i]) {
				printf("ERROR a[%d] != b[%d]  (a[i] = %d  -  b[i] = %d\n", i,i, a[i],b[i]);
				break;
			}
		}
	}

	cudaFree(d_a);
	exit(0);
}

In [ ]:
# compilatore

!nvcc -arch=sm_37 WarpSort.cu -o WarpSort
# !./WarpSort